In [1]:
import pandas as pd

data = { 
    'credit_card_number': ['1111 2222 3333 4444', '1111 2222 3333 4444','1111 2222 3333 4444',
                           '1111 2222 3333 4444'],
    'trans_datetime': ['2022-01-01 08:44', '2022-01-02 19:44', '2022-01-02 20:44', '2022-01-02 20:55'],
    'amount': [142.34, 12.34, 66.29, 112.33],
    'location': ['Sao Paolo', 'Rio De Janeiro', 'Stockholm', 'Stockholm'],
    'fraud': [False, False, True, True] 
}

df = pd.DataFrame.from_dict(data)
df['trans_datetime']= pd.to_datetime(df['trans_datetime'])
df

,credit_card_number,trans_datetime,amount,location,fraud
0,1111 2222 3333 4444,2022-01-01 08:44:00,142.34,Sao Paolo,False
1,1111 2222 3333 4444,2022-01-02 19:44:00,12.34,Rio De Janeiro,False
2,1111 2222 3333 4444,2022-01-02 20:44:00,66.29,Stockholm,True
3,1111 2222 3333 4444,2022-01-02 20:55:00,112.33,Stockholm,True


In [2]:
import hopsworks
proj = hopsworks.login()
fs = proj.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/398
Connected. Call `.close()` to terminate connection gracefully.


### Create a Feature Group

Hopsworks have comprehensive documentation on Feature Groups. Click on these links to learn more.

* [Feature Group Concept](https://docs.hopsworks.ai/3.0/concepts/fs/feature_group/fg_overview/)
* [Feature Group Creation Guide](https://docs.hopsworks.ai/3.0/user_guides/fs/feature_group/create/)
* [Feature Group API Docs](https://docs.hopsworks.ai/feature-store-api/3.0/generated/api/feature_group_api/)

In [3]:
fg = fs.get_or_create_feature_group(
     name="credit_card_transactions",
     version=1,
     description="Credit Card Transaction data",
     primary_key=['credit_card_number'],
     event_time='trans_datetime'
) 

### Write your DataFrame to the Feature Group
When you write your DataFrame to the feature group, first the DataFrame is copied to Hopsworks. 
Then a backfill ingestion job is run on Hopsworks to insert/append the DataFrame to the Feature Group. 
The job is a Spark job, and the data is stored in a Apache Hudi table in Hopsworks.

It will take about 1 minute for the ingestion job to complete.
If you don't want to wait 1 minute, you make the ingestion job run in the background with:


    fg.insert(df, write_options={"wait_for_job": False})

In [4]:
fg.insert(df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/398/fs/335/fg/933


Uploading Dataframe: 0.00% |          | Rows 0/4 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/398/jobs/named/credit_card_transactions_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f1c8febb0d0>, None)

In [5]:
query = fg.select(["amount", "location", "fraud"])

In [6]:
fv = fs.create_feature_view(name="credit_card_transactions",
                            version=1,
                            description="Features from the credit_card_transactions FG",
                            labels=["fraud"],
                            query=query)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/398/fs/335/fv/credit_card_transactions/version/1


In [ ]:
X_train, y_train, X_test, y_test = fv.train_test_split(0.5)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train

In [ ]:
y_test

### Aggregations

Compute the total amount spent on the credit card by first grouping all the rows together with the same `credit_card_number` and then summing up their amounts. 

The code first creates a new DataFrame with only the `credit_card_number` and `amount` columns, then the logic of a group-by could be described as 

    for-each (`credit_card_number`) do \sigma amount

In [7]:
df2 = df[["credit_card_number", "amount"]].groupby("credit_card_number").sum()
df2.rename(columns={"amount": "total_spent"}, inplace=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 1111 2222 3333  4444 to 1111 2222 3333 4444
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   total_spent  2 non-null      float64
dtypes: float64(1)
memory usage: 32.0+ bytes


In [8]:
df2

,total_spent
credit_card_number,
1111 2222 3333 4444,12.34
1111 2222 3333 4444,320.96


 We might also want to know at what point-in-time was that total and add a column with the datetime of the last (most recent) credit card transaction.

In [9]:
df2["as_of_datetime"] = df[["credit_card_number", "trans_datetime"]].groupby("credit_card_number").max()
df2

,total_spent,as_of_datetime
credit_card_number,,
1111 2222 3333 4444,12.34,2022-01-02 19:44:00
1111 2222 3333 4444,320.96,2022-01-02 20:55:00


The `groupby` operation sets `credit_card_number` as the index of our DataFrame.
We want `credit_card_number` as a column, as Pandas indexes are not written to the Feature Group.
We can move the index to a column using `reset_index`.

In [10]:
df2.reset_index(inplace=True)
df2

,credit_card_number,total_spent,as_of_datetime
0,1111 2222 3333 4444,12.34,2022-01-02 19:44:00
1,1111 2222 3333 4444,320.96,2022-01-02 20:55:00


We create a feature group to store the contents of `df2` with our aggregated credit card spending information.

In [11]:
fg2 = fs.get_or_create_feature_group(
     name="credit_card_spending",
     version=1,
     description="Credit Card Spending",
     primary_key=['credit_card_number'],
     event_time='as_of_datetime'
) 

In [12]:
fg2.insert(df2, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/398/fs/335/fg/934


Uploading Dataframe: 0.00% |          | Rows 0/2 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/398/jobs/named/credit_card_spending_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f1c8fe70250>, None)

Let's add some more data to our original feature group

In [13]:
more_data = { 
    'credit_card_number': ['9999 8888 7777 6666', '9999 8888 7777 6666','9999 8888 7777 6666',
                           '9999 8888 7777 6666'],
    'trans_datetime': ['2022-01-02 04:11', '2022-01-03 07:24', '2022-01-05 10:33', '2022-01-05 11:50'],
    'amount': [55.67, 84, 77.95, 183],
    'location': ['San Francisco', 'San Francisco', 'Dublin', 'Dublin'],
    'fraud': [False, False, False, False] 
}

df3 = pd.DataFrame.from_dict(more_data)
df3['trans_datetime']= pd.to_datetime(df3['trans_datetime'])

fg = fs.get_feature_group(name="credit_card_transactions", version=1)

fg.insert(df3, write_options={"wait_for_job": False})

Uploading Dataframe: 0.00% |          | Rows 0/4 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/398/jobs/named/credit_card_transactions_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f1c8feaa9d0>, None)

Now let's compute how much money was spent on the card since the last time we computed amount spent

In [14]:
df4=df

In [15]:
df4['is_big'] = df['amount'].apply(lambda amount: amount > 100)
df4

,credit_card_number,trans_datetime,amount,location,fraud,is_big
0,1111 2222 3333 4444,2022-01-01 08:44:00,142.34,Sao Paolo,False,True
1,1111 2222 3333 4444,2022-01-02 19:44:00,12.34,Rio De Janeiro,False,False
2,1111 2222 3333 4444,2022-01-02 20:44:00,66.29,Stockholm,True,False
3,1111 2222 3333 4444,2022-01-02 20:55:00,112.33,Stockholm,True,True


In [16]:
def is_small(row):
    return row['amount'] < 100

df4['is_small'] = df.apply(is_small, axis=1)

df4

,credit_card_number,trans_datetime,amount,location,fraud,is_big,is_small
0,1111 2222 3333 4444,2022-01-01 08:44:00,142.34,Sao Paolo,False,True,False
1,1111 2222 3333 4444,2022-01-02 19:44:00,12.34,Rio De Janeiro,False,False,True
2,1111 2222 3333 4444,2022-01-02 20:44:00,66.29,Stockholm,True,False,True
3,1111 2222 3333 4444,2022-01-02 20:55:00,112.33,Stockholm,True,True,False


## Time Series: Window Aggregations

Count the amount of money spent per day (make the length of the window '1d').
We will need to set the `event_time` column as the index in order to use Pandas built-in window aggregations.

In [18]:
df5 = fg.read()
df5

2022-09-15 23:26:46,674 INFO: USE `dowlingj_featurestore`
2022-09-15 23:26:47,649 INFO: SELECT `fg0`.`credit_card_number` `credit_card_number`, `fg0`.`trans_datetime` `trans_datetime`, `fg0`.`amount` `amount`, `fg0`.`location` `location`, `fg0`.`fraud` `fraud`
FROM `dowlingj_featurestore`.`credit_card_transactions_1` `fg0`


,credit_card_number,trans_datetime,amount,location,fraud
0,1111 2222 3333 4444,2022-01-02 19:44:00,66.29,Stockholm,True
1,1111 2222 3333 4444,2022-01-02 18:44:00,12.34,Rio De Janeiro,False
2,1111 2222 3333 4444,2022-01-01 07:44:00,142.34,Sao Paolo,False
3,1111 2222 3333 4444,2022-01-02 19:55:00,112.33,Stockholm,True
4,9999 8888 7777 6666,2022-01-03 06:24:00,84.00,San Francisco,False
5,9999 8888 7777 6666,2022-01-05 10:50:00,183.00,Dublin,False
6,9999 8888 7777 6666,2022-01-02 03:11:00,55.67,San Francisco,False
7,9999 8888 7777 6666,2022-01-05 09:33:00,77.95,Dublin,False


In [19]:
df5 = df5.set_index('trans_datetime')

In [21]:
df5 = df5.sort_index()

In [22]:
max = df5.rolling('1D').amount.max()
max

trans_datetime
2022-01-01 07:44:00    142.34
2022-01-02 03:11:00    142.34
2022-01-02 18:44:00     55.67
2022-01-02 19:44:00     66.29
2022-01-02 19:55:00    112.33
2022-01-03 06:24:00    112.33
2022-01-05 09:33:00     77.95
2022-01-05 10:50:00    183.00
Name: amount, dtype: float64

In [23]:
mean = df5.rolling('1D').amount.mean()
mean

trans_datetime
2022-01-01 07:44:00    142.340000
2022-01-02 03:11:00     99.005000
2022-01-02 18:44:00     34.005000
2022-01-02 19:44:00     44.766667
2022-01-02 19:55:00     61.657500
2022-01-03 06:24:00     68.740000
2022-01-05 09:33:00     77.950000
2022-01-05 10:50:00    130.475000
Name: amount, dtype: float64

In [24]:
by_cc_mean = df5.groupby('credit_card_number').amount.rolling('1D').count()
by_cc_mean

credit_card_number    trans_datetime     
1111 2222 3333  4444  2022-01-02 18:44:00    1.0
1111 2222 3333 4444   2022-01-01 07:44:00    1.0
                      2022-01-02 19:44:00    1.0
                      2022-01-02 19:55:00    2.0
9999 8888 7777 6666   2022-01-02 03:11:00    1.0
                      2022-01-03 06:24:00    1.0
                      2022-01-05 09:33:00    1.0
                      2022-01-05 10:50:00    2.0
Name: amount, dtype: float64